In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient


In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])
client = EfdClient('summit_efd')

In [ ]:
expId = 2024101700002
mData = butler.get('raw.metadata', detector=0, exposure=expId)


In [ ]:
expId = 2024101700002
mData = butler.get('raw.metadata', detector=0, exposure=expId)
tStart = mData['DATE-BEG']
tEnd = mData['DATE-END']
startExp = Time(tStart, scale='tai').utc
endExp = Time(tEnd, scale='tai').utc

t_start = startExp - TimeDelta(5.0, format='sec')
t_end = endExp + TimeDelta(5.0, format='sec')
mount_position = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                            ['azimuthCalculatedAngle', 'elevationCalculatedAngle'], t_start.utc, t_end.utc)
mount_raw = await client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                            ['*'], t_start.utc, t_end.utc)

inPos = await client.select_time_series("lsst.sal.ATMCS.logevent_allAxesInPosition", \
                            ['*'], t_start, t_end)
inPos = inPos[inPos['inPosition'] == True]
print(len(inPos))
print(len(mount_raw))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,5))
mount_position['azimuthCalculatedAngle'].plot(ax=ax)
for time in inPos.index:
    ax.axvline(time, ls='--', color='black', label="AllAxesInPos")
ax.axvline(startExp.isot, ls='--', color='green', label="Start Exposure")
ax.axvline(endExp.isot, ls='--', color='red', label="End Exposure")
ax.set_ylabel("Azimuth(deg)")
ax.set_title(f"Image {expId} Python ATMCS CSC")
plt.legend(loc='lower left')
#plt.savefig(f"/home/cslage/DATA/Python_ATMCS_{expId}_17Oct24.png")

In [ ]:
azs = []
for j in range(3):
    for i in range(100):
        azs.append(mount_raw[f"azimuthCalculatedAngle{i}"][j])
    

In [ ]:
plt.plot(azs, marker='x')
plt.title(f"Raw Azimuth Data {expId}")
plt.ylabel("Azimuth(deg)")
plt.savefig(f"/home/cslage/DATA/Python_ATMCS_Raw_Azimuth_{expId}_17Oct24.png")